# Báo cáo Đồ án Cuối kì - Lập trình Song song
##### Nhóm 9:
##### Đoàn Thị Minh Anh - 22120213
##### Trần Hoàng Kim Ngân - 22120224

## 1. Mô tả bài toán:
### 1.1. Phát biểu bài toán:
- Xác định bài toán phân loại ảnh:  
  - Bài toán: Học đặc trưng không giám sát từ bộ dữ liệu ảnh CIFAR-10 bằng Autoencoder rồi dùng ác đặc trưng đã học để huấn luyện một bộ phân loại SVM cho nhiệm vụ phân loại 10 lớp.
  - Quy trình:
    - Huấn luyện Autoencoder trên toàn bộ ảnh huấn luyện không dùng nhãn để tối thiểu hóa loss MSE. Encoder xuất ra vector đặc trưng có kích thước 8192 cho mỗi ảnh.
    - Dùng encoder đã huấn luyện để trích xuất đặc trưng cho tập huấn luyện và tập kiểm tra, rồi huấn luyện SVM sử dụng các nhãn lớp.
    - Đánh giá hiệu năng SVM.  
  - Mục tiêu: Chứng minh rằng đặc trưng học được từ Autoencoder giúp phân loại hiệu quả và triển khai CUDA để chạy autoencoder trên GPU hỗ trợ tăng tốc đáng kể so với chạy bằng CPU.
- Mô tả động lực tăng tốc GPU:
  - Bài toán đòi hỏi xử lý một lượng lớn phép tính số học, đặc biệt trong các tầng tích chập (Convolution), phép nhân ma trận và lan truyền ngược. Việc triển khai các phép tính này trên CPU theo tuần tự thường rất mất thời gian và kém hiệu quả. Trong khi đó, các phép tính này đều có khả năng song song hóa cao, hoàn toàn có thể triển khai một cách hợp lý để ứng dụng GPU.
  - Bộ dữ liệu CIFAR-10 với 50,000 ảnh train và 10,000 ảnh test đòi hỏi khả năng truy xuất bộ nhớ mạnh mẽ. Tận dụng GPU tốt sẽ giúp hệ thống xử lý dữ liệu hiệu quả hơn, đạt tốc độ huấn luyện nhanh hơn và cải thiện chất lượng đặc trưng thu được cho nhiệm vụ phân loại hình ảnh.
### 1.2. Tổng quan về bộ dữ liệu CIFAR-10:
- Đặc tả tập dữ liệu (kích thước, lớp, phân chia):
  - Kích thước ảnh: 32x32 pixels (RGB).
  - 10 lớp gồm: airplane,automobile, bird, cat, deer, dog, frog, horse, ship, truck.
  - Training set: 50,000 ảnh (5,000 ảnh/lớp).
  - Test set: 10,000 ảnh (1,000 ảnh/lớp).
  - Tổng số ảnh: 60,000 ảnh
  - Format: File nhị phân với giá trị pixel uint8.
- Hiển thị hình ảnh mẫu từ mỗi lớp (visualization):
![Hình minh họa CIFAR-10](https://drive.google.com/uc?export=view&id=1Gy6BtsEWLgvlRHhyh5M7H5PCLy76H9Ss)

- Giải thích các bước tiền xử lý dữ liệu (normalization, format):
  - Chuyển đổi kiểu dữ liệu: Dữ liệu pixel gốc trong file ảnh là uint8, là số nguyên 8-bit không dấu nằm trong khoảng [0,255]. Để tiện cho việc chuẩn hoá các giá trị về khoảng [0,1], ở đây chúng em sẽ chuyển chúng về kiểu float.
  - Chuyển về [0,1]: Sau khi chuyển sang kiểu số thực, các giá trị pixel sẽ được chuẩn hoá về khoảng [0,1]. Việc này giúp mạng nơ-ron hội tụ nhanh hơn và hoạt động ổn định hơn.
### 1.3. Kiến trúc Autoencoder:
- Mô tả kiến ​​trúc mạng bằng sơ đồ

![Hình sơ đồ kiến trúc mạng](https://drive.google.com/uc?export=view&id=1TZZe0q-VkPQI0rTmbzO1prhDyyeocfeE)
- Xác định kích thước và phép biến đổi lớp
| Khối | Lớp | Tham số chính | Input shape | Output shape |
|---|---|---|---|---|
| Input | - | - | (32, 32, 3) | (32, 32, 3) |
| Encoder | Conv2D + ReLU | 256 filters, K=3, pad=1, stride=1 | (32, 32, 3) | (32, 32, 256) |
| Encoder | MaxPool2D | 2×2, stride=2 | (32, 32, 256) | (16, 16, 256) |
| Encoder | Conv2D + ReLU | 128 filters, K=3, pad=1, stride=1 | (16, 16, 256) | (16, 16, 128) |
| Encoder | MaxPool2D | 2×2, stride=2 | (16, 16, 128) | **(8, 8, 128)** |
| Latent | Flatten (khi trích đặc trưng) | 8×8×128 = 8192 | (8, 8, 128) | **(8192, )** |
| Decoder | Conv2D + ReLU | 128 filters, K=3, pad=1, stride=1 | (8, 8, 128) | (8, 8, 128) |
| Decoder | UpSample2D | scale=2 | (8, 8, 128) | (16, 16, 128) |
| Decoder | Conv2D + ReLU | 256 filters, K=3, pad=1, stride=1 | (16, 16, 128) | (16, 16, 256) |
| Decoder | UpSample2D | scale=2 | (16, 16, 256) | (32, 32, 256) |
| Output | Conv2D | 3 filters, K=3, pad=1, stride=1 | (32, 32, 256) | (32, 32, 3) |

- Giải thích cấu trúc bộ mã hóa-giải mã và biểu diễn latent
  - Encoder (downsampling path): Hai lớp Conv+ReLU giúp học đặc trưng cục bộ (biên, texture) còn MaxPool giúp nèn thông tin và tăng tính bất biến theo dịch chuyển. Kết quả cuối encoder là tensor (8, 8, 128). Khi trích đặc trưng để phân loại, tensor này được flatten thành vector 8192 chiều.
  - Latent space: là nút thắt cổ chai chưa thông tin quan trọng nhất để tái tạo ảnh. Latent càng tốt thì ảnh reconstruct càng gần ảnh gốc.
  - Decoder (upsampling path): đối xứng với encoder, dùng UpSample để đảo quá trình downsample và Conv nhằm tái tạo ảnh về (32, 32, 3). Lớp cuối không dùng activation, thườngđể mô hình tự học giá trị tái tạo.
### 1.4. Mục tiêu đồ án:
- Mục tiêu hiệu suất (thời gian luyện tập, mục tiêu
tăng tốc, độ chính xác): Đồ án hướng đến xây dựng một hệ thống trích chọn đặc trưng và phân loại ảnh hiệu quả trên tập dữ liệu CIFAR-10. Các mục tiêu hiệu suất chính gồm:
  - Giảm thời gian huấn luyện bằng cách tận dụng GPU và tối ưu các bước tính toán quan trọng trong Autoencoder.
  - Đạt được tốc độ tăng tốc (speedup) đáng kể so với phiên bản chạy trên CPU.
  - Đạt độ chính xác phân loại thỏa yêu cầu tối hiểu (khoảng 55-65%).
- Mục tiêu học tập kỹ thuật: Đồ án giúp nhóm phát triển kiến thức, kỹ năng liên quan đến học máy và kỹ năng lập trình song song trên GPU, cụ thể:
  - Hiểu và triển khai được kiến trúc Autoencoder cho bài toán học đặc trưng không giám sát.
  - Nắm vững quy trình huấn luyện mô hình học sâu (forward, backward, tối ưu, đánh giá).
  - Thực hành lập trình song song trên GPU bằng CUDA.
  - Tối ưu hóa các tác vụ tính toán như biến đổi tensor, tích chập, nhân ma trận nhằm khai thác tối đa khả năng song song của GPU.
- Tiêu chí thành công
  - Chức năng đúng: Autoencoder huấn luyện thành công, tái tạo ảnh hợp lý và bộ phân loại hoạt động chính xác.
  - Trích chọn đặc trưng hiệu quả: đặc trưng nén từ Autoencoder giúp cải thiện độ chính xác phân loại so với baseline.
  - Tăng tốc rõ rệt khi chuyển từ CPU sang GPU, với báo cáo định lượng (speedup, thời gian huấn luyện).
  - Kết quả thực nghiệm đầy đủ bao gồm biểu đồ loss, accuracy, tốc độ huấn luyện, hình ảnh tái tạo, confusion matrix.
  - Báo cáo hoàn chỉnh, phân tích hợp lý và giải thích được tác động của GPU trong toàn hệ thống.

## 2. Các giai đoạn triển khai:


### 2.1. CPU cơ bản
#### Mục tiêu:
- Mục tiêu nhóm muốn đạt được trong giai đoạn này:
  - Xây dựng một phiên bản hoàn chỉnh của autoencoder trên CPU để làm baseline tham chiếu cho các phiên bản GPU sau này.
  - Triển khai đầy đủ các layer cần thiết (Convolution, ReLU, MaxPooling, Upsampling, MSE Loss) cùng forward và backward pass trên CPU để đảm bảo mô hình có thể train được và đạt được reconstruction loss hợp lý.
  - Đo lường thời gian huấn luyện và hiệu suất ban đầu trên CPU, từ đó xác định rõ các điểm nghẽn hiệu năng (bottleneck) để làm cơ sở cho việc tối ưu hóa trên GPU ở các phase tiếp theo.
  - Có thể tạo ra một phiên bản đúng và có thể chạy được để dễ dàng so sánh kết quả với các phiên bản GPU khác và làm nền tảng cho việc chuyển đổi sang code CUDA.
- Tại sao giai đoạn này là cần thiết:
  - Kiểm chứng độ chính xác: Lập trình song song trên GPU vốn phức tạp với các vấn đề về đồng bộ luồng và quản lý bộ nhớ. Khi kết quả đầu ra bị sai lệch, rất khó để phân định nguyên nhân đến từ lỗi logic thuật toán hay lỗi kỹ thuật song song. Phiên bản CPU hoạt động như một chân lý (Ground Truth) tin cậy. Trong các giai đoạn sau, kết quả từ GPU sẽ được so sánh trực tiếp với CPU để đảm bảo tính đúng đắn trước khi tính đến hiệu năng.
  - Nhận diện điểm nghẽn: Việc chạy trên CPU giúp xác định lớp nào (Convolution, Pooling hay Upsampling) chiếm nhiều thời gian xử lý nhất. Đây là chỉ dẫn quan trọng để biết nên dồn lực tối ưu vào lớp nào khi chuyển sang GPU.
  - Thấu hiểu sâu sắc kiến trúc giải thuật: Việc chuyển đổi trực tiếp một công thức toán học sang CUDA kernel là rất rủi ro nếu chưa hiểu rõ luồng dữ liệu. Việc tự tay cài đặt các lớp phức tạp như Convolution hay MaxPooling bằng các vòng lặp tường minh giúp lập trình viên nắm bắt tường tận cơ chế truy cập bộ nhớ, cách xử lý biên và sải bước (stride).
#### Chi tiết triển khai:
- Data Pipeline: Cách tải và tiền xử lý dữ liệu CIFAR-10

  Chúng em tải bộ dữ liệu CIFAR-10 dạng binary nguyên bản từ trang web chính thức của tác giả.

  Bộ dữ liệu gồm 5 file training batch (`data_batch_1.bin` đến `data_batch_5.bin`) và 1 file test batch (`test_batch.bin`).

  Mỗi bản ghi trong các file có đúng *3073* bytes, được sắp xếp như sau:  
  - Byte 0: nhãn lớp (1 byte, giá trị 0-9).
  - Byte 1-1024: toàn bộ kênh màu đỏ (1024 pixel).
  - Byte 1025-2048: toàn bộ kênh màu xanh lá (1024 pixel).
  - Byte 2049-3072: toàn bộ kênh màu xanh lam (1024 pixel).

  Quy trình đọc và tiền xử lý dữ liệu được thực hiện như sau:

  - Mở và đọc lần lượt từng file binary theo đúng thứ tự.
  - Với mỗi bản ghi 3073 bytes, tách riêng 1 byte nhãn và 3072 byte pixel.
  - Chuyển ngay 3072 byte pixel từ kiểu `uint8` sang `float` trong quá trình đọc, đồng thời giữ nguyên thứ tự kênh R, G và B như file gốc.
  - Lưu toàn bộ dữ liệu ảnh train (50.000 ảnh) và test (10.000 ảnh) vào các mảng float liên tục trong bộ nhớ host với kích thước là *N * 3072*.
  - Sau khi tải xong, thực hiện chuẩn hóa một lần duy nhất bằng cách chia toàn bộ giá trị pixel cho 255.0 để đưa về khoảng [0.0, 1.0].
  - Để hỗ trợ mini-batch training, chúng em duy trì một mảng chỉ số `train_indices[50000]`. Ở đầu mỗi epoch, hàm `shuffle_cifar10()` sử dụng thuật toán Fisher-Yates shuffle để xáo trộn ngẫu nhiên thứ tự ảnh.
  - Hàm `get_next_batch()` sao chép nhanh dữ liệu batch từ `train_images` vào bộ nhớ đích bằng `memcpy`, đảm bảo hiệu suất cao và dễ dàng chuyển đổi sang GPU ở các phase sau.
- Mô tả cách triển khai các lớp (Convolution, ReLU, MaxPool, UpSample):
  1) Convolution:
  - Forward: Hàm `Conv2D_Forward` thực hiện phép tích chập chuẩn với padding = 1, stride = 1, kernel 3x3. Duyệt theo thứ tự NCHW, hỗ trợ nhiều input và output channels. Bias được cộng trực tiếp sau tích chập.
  - Backward: Tương tự với phiên bản forward, backward cũng thực hiện duyệt mảng theo thứ tự NCHW.
    - Input: Hàm `Conv2D_Backward_Input` thực hiện full convolution giữa gradient từ lớp sau (d_output) với kernel đã được xoay 180°. Kết quả được cộng dồn vào đúng vị trí tương ứng trên input gradient.
    - Weight: Hàm `Conv2D_Backward_Kernel` thực hiện tích chập giữa input của lớp hiện tại và gradient từ lớp sau (d_output), sau đó cộng dồn vào ma trận gradient trọng số.
    - Bias: Hàm `Conv2D_Backward_Biases` thực hiện với mỗi filter, gradient của bias bằng tổng tất cả các giá trị gradient tương ứng trên toàn bộ feature map đầu ra (d_output).
  2) ReLU:
  - Forward: Với từng phần tử, thực hiện so sánh giá trị input[i], với i là index hiện tại trong mảng đầu vào, với 0.0f nhằm đảm bảo đầu ra luôn dương.
  - Backward: Hàm `Relu_Backward` nhân gradient từ lớp sau với 1 nếu input > 0, ngược lại bằng 0, đảm bảo gradient chỉ lan truyền qua các neuron dương.
  3) MaxPool:
  - Forward: Hàm `MaxPool2D_Forward` thực hiện pooling 2x2, stride = 2, giảm kích thước không gian xuống một nửa. Với mỗi vị trí output, tìm giá trị lớn nhất trong vùng 2x2 của từng kênh.
  - Backward: Hàm `MaxPool2D_Backward` chỉ truyền gradient về đúng vị trí có giá trị max trong forward pass và các vị trí còn lại nhận giá trị 0.
  4) UpSample:
  - Forward: Hàm `UpSample2D_Forward` thực hiện nearest-neighbor upsampling với scale_factor = 2. Mỗi pixel đầu vào được sao chép vào khối 2x2 tương ứng trên output.
  - Backward: Hàm `UpSample2D_Backward` cộng dồn gradient từ toàn bộ khối 2x2 của output về đúng pixel gốc trên input.
- Vòng lặp huấn luyện được cấu trúc như sau:
  1) Lặp qua các epoch: Chúng em huấn luyện trong 20 epoch.
  2) Xáo trộn dữ liệu: Ở đầu mỗi epoch, hàm `shuffle_cifar10(&data)` được gọi để áp dụng thuật toán Fisher-Yates shuffle trên mảng `train_indices`, đảm bảo thứ tự các ảnh được thay đổi ngẫu nhiên, giúp mô hình học tốt hơn và tránh bias theo thứ tự file.
  3) Xử lý từng mini-batch:
  - Số batch mỗi epoch được tính tự động bằng công thức: `num_batches = train_subset_size / batch_size`, ở đây do có những giới hạn về bộ nhớ, nên chúng em chỉ gán `train_subset_size = 1000` (chỉ train với 1000 ảnh). Ngoài ra, nhóm áp dụng `batch_size = 32`.
  - Với mỗi batch_id, hàm `get_next_batch()` sao chép nhanh dữ liệu từ `train_images` (theo thứ tự đã shuffle) vào bộ nhớ tạm `batch_images` bằng `memcpy`.
  - Dữ liệu batch được copy vào batch_input của struct autoencoder (buffer đã được cấp sẵn trong struct CPUAutoEncoder) để các hàm forward/backward có thể truy cập trực tiếp.
  4) Quy trình huấn luyện một batch:
  - Forward pass: Gọi hàm dành cho forward để tính lần lượt output của tất cả các layer từ input đến reconstructed output.
  - Tính loss: Sử dụng hàm để tính Mean Squared Error giữa ảnh gốc (batch_input) và ảnh tái tạo (final_output). Loss của batch hiện tại được hiển thị trên terminal.
  - Backward pass: Gọi hàm dành cho backward để tính gradient cho tất cả các layer theo thứ tự ngược lại.
  - Cập nhật tham số: Gọi hàm cập nhật để áp dụng Gradient Descent với `learning rate = 0.001`, cập nhật trọng số và bias của tất cả các lớp Convolution.
- Đoạn mã chính: Hiển thị 2-3 hàm quan trọng (ví dụ: convolution function structure và cấu trúc vòng lặp chính)
#### Kết quả:
- Thời gian huấn luyện cho mỗi epoch và tổng thời gian huấn luyện:
  - Với mỗi epoch, phiên CPU sẽ thực thi trong khoảng 489 giây.
  - Tổng thời gian thực thi 20 epoch là khoảng 9780 giây (tương đương gần với 2 giờ 43 phút).
- Final reconstruction loss: 0.054788.
- Sample reconstructed images (show original vs reconstructed)
![Ảnh mẫu reconstructed bởi CPU](https://drive.google.com/uc?export=view&id=1HPJctlPGdm7VdnxSwX7QfXSTkSVR500y)
- Memory usage
#### Những điểm chính:
- Học được gì từ thuật toán
- Những hiểu biết sâu sắc đã hướng dẫn việc triển khai GPU:
  - Hiểu rõ hoạt động các lớp như Convolution, MaxPool, UpSample, giúp cho việc thực hiện song song hoá trở hơn hiệu quả hơn, dễ dàng hình dung được luồng làm việc của GPU.
  - Ở forward pass, có thể thấy mảng kernel 3x3 sẽ không thay đổi trong suốt quá trình tính toán, đồng thời kích thước lớn nhất của kernel khoảng 9 KB (< 64 KB, kích thước giới hạn của Constant Memory). Từ đó, có thể xác định việc dùng Constant Memory cho mảng này.
  -

### 2.2. GPU cơ bản
#### Mục tiêu:
- Chuyển việc triển khai trên CPU sang trên GPU bằng song song hóa cơ bản cho các phép toán chính là Con2D, MaxPool2D, ReLU và UpSample2D.
- So sánh output GPU với CPU để đảm bảo kết quả đúng so với CPU.
- Thiết lập baseline hiệu suất GPU: đo thời gian thực thi theo kernel và tổng thời gian để làm mốc cho các version tối ưu phía sau.
#### Chi tiết cài đặt:
- Chiến lược song song hóa:
  - Layout dữ liệu: Tensor được lưu dạng 1D liên tục theo NCHW với công thức tỉ số là
  $$idx(n,c,h,w) = ((n \cdot C + c)\cdot H +h)\cdot W +w$$
  - Các kernel được thiết kế theo nguyên tắc:
    - Các phép toán tạo output 4D như Conv/Pool/Upsample: 1 thread xử lý 1 phần tử output.
    - Các phép toán "element-wise" như ReLU: 1 thread xử lý 1 phần tử.
  - Cách ánh xạ thread:
    - blockIdx.x và blockIdx.y quét theo không gian w và h.
    - blockIdx.z gộp (N,C) hoặc (N, C_out) bằng chỉ số nc.
- Kernel Design:
  - Convolution kernel: thread-to-output mapping
    - Ý tưởng: Mỗi thread tính đúng 1 phần tử output Y[n, c_out, h_out, w_out]
    -
  - Pooling kernel: Mỗi thread tính Y[n, c_out, h_out, w_out] bằng max của 4 phần tử trong cửa sổ 2x2.
  - Upsample kernel: Mỗi thread ghi 1 pixel output, lấy từ output bằng phép chia nguyên.
  - ReLU kernel (element-wise): Mỗi thread xử lý một phần tử.
- Chiến lược quản lý bộ nhớ device: Sử dụng chiến lược cấp phát trước (pre-allocation/static). Toàn bộ bộ đệm trên device cho tham số mô hình (weights/biases), các tensor trung gian (activations) và gradient tương ứng được cudaMalloc một lần duy nhất trong hàm gpu_autoencoder_init() trước khi bắt đầu huấn luyện. Trong quá trình train, mỗi iteration chỉ cần copy input batch từ host sang device và tái sử dụng các buffer đã cấp phát cho các bước forward/backward, nhờ đó tránh việc cấp phát/giải phóng lặp lại trong vòng lặp huấn luyện.
- Key Code Snippets:
  - Hàm idx4():
  ```
  __device__ __host__ inline int idx4(int n, int c, int h, int w, int C, int H, int W)
  {
    return ((n * C + c) * H + h) * W + w;
  }
  ```
  - Hàm
#### Results:
- Training time per epoch and total training time
- Speedup over CPU baseline (include table and chart)
- GPU memory usage
- Sample reconstructed images
![Ảnh mẫu reconstructed bởi GPU naive](https://drive.google.com/uc?export=view&id=1Phzzp3KKKRkMTYVaK5ykaM0lLNFVn5-o)
- Verification that outputs match CPU (show error metrics)
#### Profiling Analysis:
- Basic profiling results (time spent in each kernel type)
- Memory bandwidth utilization (if measured)
- Initial bottleneck identification
#### Key Takeaways:
- What was surprisingly fast or slow?
- Where do you see optimization opportunities?


### 2.3. GPU Optimized Implementation - Version 1
#### Optimization Focus:
#### Objectives:
- What specific optimization(s) you targeted
- Expected performance improvement
#### Implementation Details:
- Optimization Techniques Applied
  - Detailed explanation of the optimization (e.g., shared memory tiling)- Why this optimization should help
  - Implementation approach
- Key Code Snippets: Show the optimized kernel or key changes
#### Results:
- Training time comparison with previous version
- Speedup over previous phase (incremental and cumulative)
- Performance metrics (bandwidth utilization, occupancy)
- Sample reconstructed images
![Ảnh mẫu reconstructed bởi GPU version 1](https://drive.google.com/uc?export=view&id=1HxMYgUUDyIy5BUthej5hNJIPSvblqDiS)
- Profiling comparison: before vs after
#### Analysis:
- Why did this optimization work (or not work as expected)?
- What did profiling reveal?
- What's the next bottleneck?
#### Key Takeaways:
- Lessons learned from this optimization
- Applicability to other problems

### 2.4. GPU Optimized Implementation - Version 2
#### Optimization Focus:
#### Objectives:
- What specific optimization(s) you targeted
- Expected performance improvement
#### Implementation Details:
- Optimization Techniques Applied
  - Detailed explanation of the optimization (e.g., shared memory tiling)- Why this optimization should help
  - Implementation approach
- Key Code Snippets: Show the optimized kernel or key changes
#### Results:
- Training time comparison with previous version
- Speedup over previous phase (incremental and cumulative)
- Performance metrics (bandwidth utilization, occupancy)
- Sample reconstructed images
![Ảnh mẫu reconstructed bởi GPU version 2](https://drive.google.com/uc?export=view&id=1CgsqohZwarREM4Fmc1W5ynOcQRsZEYL1)
- Profiling comparison: before vs after
#### Analysis:
- Why did this optimization work (or not work as expected)?
- What did profiling reveal?
- What's the next bottleneck?
#### Key Takeaways:
- Lessons learned from this optimization
- Applicability to other problems

### 2.5. SVM Integration
#### Objectives:
- Extract features using trained encoder
- Train SVM classifier on learned features
- Evaluate end-to-end classification performance
#### Implementation Details:
- Feature Extraction: How you extracted 1,024-dim features from encoder
- LIBSVM Integration: How you interfaced with LIBSVM
- Hyperparameter Selection: SVM parameters chosen (C, gamma, kernel)
- Key Code Snippets: Feature extraction and SVM training code
#### Results:
- Feature extraction time (50K train + 10K test)
- SVM training time
- Classification accuracy on test set
- Per-class accuracy breakdown (table)
- Confusion matrix (visualization)
- Comparison with baseline methods (if available)
#### Analysis:
- Which classes are easiest/hardest to classify?
- What does the confusion matrix reveal?
- How does accuracy compare to expectations?
#### Key Takeaways:
- Quality of learned features
- Effectiveness of two-stage approach

## 3. Comprehensive Performance Analysis
### 3.1. Performance Comparison Across All Phases:
#### Table Format:
| Phase        | Training Time | Speedup (vs CPU) | Incremental Speedup | Memory Usage | Key Optimization        |
|-------------|---------------|------------------|----------------------|-------------|-------------------------|
| CPU Baseline| 1800s         | 1.0×             | -                    | -           | -                       |
| GPU Basic   | 180s          | 10.0×            | 10.0×                | 2.1 GB      | Parallelization         |
| GPU Opt v1  | 45s           | 40.0×            | 4.0×                 | 2.3 GB      | Shared memory           |
| GPU Opt v2  | 25s           | 72.0×            | 1.8×                 | 2.5 GB      | Kernel Fusion + Streams |

#### Visualizations:
- Bar chart showing training time across phases
- Line graph showing cumulative speedup

## 4. Bài học và khó khăn trong quá trình thực hiện:
### 4.1. Những kỹ thuật quan trọng:
- Lập trình CUDA:
  - Hiểu rõ cách tổ chức kernel theo mô hình grid/
### 4.2. Khó khăn chính và giải pháp:
#### - Challenge 1:
- Problem:
- Solution:
- Lesson:
#### - Challenge 2:
- Problem:
- Solution:
- Lesson:

## 5. Conclusion and Future Work:
### 5.1. Project Summary:
- Recap of what was accomplished
- Final performance metrics summary table
- Achievement of original objectives
### 5.2. Key Achievements:
Highlight your best results:
- Maximum speedup achieved
- Classification accuracy
- Most successful optimization
- Technical skills mastered
### 5.3. Limitations:
Honestly discuss:
- Current performance bottlenecks
- Accuracy limitations
- Implementation constraints
### 5.4. Future Improvements:

